In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
from constrained_foopsi import *
import scipy.io
import time
drive_path = '/Volumes/Brain2016'
manifest_path = os.path.join(drive_path,'BrainObservatory/manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)

# get all good cells and pick from there
good=np.load('good_dfof3_cell_ids.npy')

In [22]:
### THIS IS THE LINE TO MODIFY FOR DIFFERENT STIMULUS
stimulus_type='drifting_gratings'

# Different celltypes
L23_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[175], cre_lines=['Cux2-CreERT2'],stimuli=[stimulus_type])
L4_Cux2_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'], imaging_depths=[275], cre_lines=['Cux2-CreERT2'],stimuli=[stimulus_type])
L4_Rorb_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Rorb-IRES2-Cre'],stimuli=[stimulus_type])
L4_Scnn1a_VISp=boc.get_ophys_experiments(targeted_structures=['VISp'],  cre_lines=['Scnn1a-Tg3-Cre'],stimuli=[stimulus_type])

In [27]:
# get one celltype to pick data
cell_line='L4_Scnn1a_VISp'

i=0
matlabdict={}
for experiment in eval(cell_line):
    data=boc.get_ophys_experiment_data(ophys_experiment_id = experiment['id'])
    #select good cells with dfof >3%
    cells=data.get_cell_specimen_ids()
    good_cells=[n for n in cells if n in good]
    timestamps,exp_dff=data.get_dff_traces(cell_specimen_ids=good_cells)
    running_speed,_ = data.get_running_speed()
    print len(timestamps),len(exp_dff),len(running_speed)   
### THIS IS THE LINE TO MODIFY FOR DIFFERENT STIMULUS
    stim_time=data.get_stimulus_table(stimulus_type).sort_values(by=['blank_sweep','temporal_frequency','orientation']).reset_index(drop=True)

    time_rearrange=np.empty([0])
    running_rearrange=np.empty([0])
#    chunksize=stim_time['end'][0]-stim_time['start'][0]
    chunksize=60
    dfof_rearrange=np.empty([len(good_cells),0])
    ######rearrange data according to the stimulus
    for section in range(stim_time.shape[0]):
        time_chunk=timestamps[stim_time['start'][section]:stim_time['start'][section]+chunksize]
        time_rearrange=np.concatenate((time_rearrange,time_chunk))
        exp_dff_chunk=exp_dff[:,stim_time['start'][section]:stim_time['start'][section]+chunksize]
        dfof_rearrange=np.concatenate((dfof_rearrange,exp_dff_chunk),axis=1)
        running_chunk=running_speed[stim_time['start'][section]:stim_time['start'][section]+chunksize]
        running_rearrange=np.concatenate((running_rearrange,running_chunk))
    
    print time_rearrange.shape,dfof_rearrange.shape,running_rearrange.shape
    matlabdict[cell_line+str(i)]=dfof_rearrange
    matlabdict['running_speed'+str(i)]= running_rearrange
    matlabdict['timestamps_'+str(i)]= time_rearrange
    i=i+1

115503 60 115503
(37680,) (60, 37680) (37680,)
115459 111 115459
(37680,) (111, 37680) (37680,)
115495 191 115495
(37680,) (191, 37680) (37680,)
115477 117 115477
(37680,) (117, 37680) (37680,)


In [28]:
savingfile='/Volumes/Brain2016/modelling/'+ stimulus_type+ '/'+cell_line
print savingfile
scipy.io.savemat(savingfile,matlabdict)

/Volumes/Brain2016/modelling/drifting_gratings/L4_Scnn1a_VISp
